# Read Data

In [1]:
import csv
import pandas
import numpy as np

dfApp = pandas.read_csv('data/APP_new_test.csv')
dfArt = pandas.read_csv('data/testUADwell.csv')
dfUser = pandas.read_csv('data/user_test.csv')
dfArtCT = pandas.read_csv('data/test_train_articleID_codetable.csv')

In [2]:
article = dfArtCT.article_code.tolist()
aid = dfArtCT.article_id.tolist()
user = dfUser.user_code.tolist()
uid = dfUser.user_id.tolist()    

In [6]:
#delete the column names and userids/usercodes, only keep the freq values
Uapp = dfApp.values
Uapp = np.delete(Uapp, np.s_[0:2], 1)

#delete the column names and userids/usercodes, only keep the dwelltime values
#     dfArt.dropna(axis=1, how='all')
Uart = dfArt.values
Uart = np.delete(Uart, 0, 1)

print Uapp
print Uart

[[  0.           0.           0.         ...,  16.66666667   0.           0.        ]
 [  0.           0.           0.         ...,  16.66666667  15.625       62.5       ]
 [  0.           0.           0.         ...,   0.          31.25
   20.83333333]
 ..., 
 [  0.           0.           0.         ...,   0.           7.8125       0.        ]
 [  0.           0.           0.         ...,   0.           0.           0.        ]
 [  0.           0.           0.         ...,   0.          15.625        0.        ]]
[[ nan  nan  nan ...,  nan  nan  nan]
 [ nan  nan  nan ...,  nan  nan  nan]
 [ nan  nan  nan ...,  nan  nan  nan]
 ..., 
 [ nan  nan  nan ...,  nan  nan  nan]
 [ nan  nan  nan ...,  nan  nan  nan]
 [ nan  nan  nan ...,  nan  nan  nan]]


In [5]:
# #to speed up, only use the top-20 to predict
# user = user[:20]
# uid = uid[:20]
# Uapp = Uapp[:20]
# # Uart = Uart[:20]

# # print(Uapp,Uart)

In [43]:
#delete articles with all nan values
dfArtP = dfArt.dropna(axis=1, how='all')
# articleP = list(dfArtP)
Uartp = dfArtP.values
Uartp = np.delete(Uartp, 0, 1)
# Uartp = Uartp[:20]
print Uartp

[[ nan  nan  nan ...,  nan  nan  nan]
 [ nan  nan  nan ...,  nan  nan  nan]
 [ nan  nan  nan ...,  nan  nan  nan]
 ..., 
 [ nan  nan  nan ...,  nan  nan  nan]
 [ nan  nan  nan ...,  nan  nan  nan]
 [ nan  nan  nan ...,  nan  nan  nan]]


In [41]:
print len(Uartp[0])

31655


# Proposed Model: Collective Matrix Factorization

In [7]:
# -*- coding: utf-8 -*-
"""
Created on Tue Nov 14 16:10:11 2017
@author: Mika
"""

import numpy.linalg as LA
###############################################################################

"""
@INPUT:
    R     : a matrix to be factorized, dimension N x M
    I     : a matrix to be added,dimension N x C
    P     : an initial matrix of dimension N x K
    Q     : an initial matrix of dimension M x K
    S     : an initial matrix of dimension C x K
    K   : the number of latent features for R and I
    R=P * Q;
    I=P * S；
    steps : the maximum number of steps to perform the optimisation
    alpha : the learning rate
    beta  : the regularization parameter
@OUTPUT:
    the final matrices P and Q and S
"""    
def collective_matrix_factorization(R, I, P, Q, S, K, steps=5000, lamda=0.6, alpha=0.0002, beta=0.02):
    Q = Q.T
#     if S!=None:
    S = S.T        
    for step in range(steps):
        print(step)
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    eij = R[i][j] - cp.dot(P[i,:],Q[:,j])
#                     if I!=None:#have transfer learning#
                    for x in range(len(I[i])):
                        if I[i][x] > 0:
                            eix= I[i][x] - cp.dot(P[i,:],S[:,x])
                            for k in range(K):
                                P[i][k] = P[i][k] + alpha * (2 * lamda * eij * Q[k][j] + 2 * (1 - lamda) * eix * S[k][x] - beta * P[i][k])
                                Q[k][j] = Q[k][j] + alpha * (2 * lamda * eij * P[i][k] - beta * Q[k][j])
                                S[k][x] = S[k][x] + alpha * (2 * (1-lamda) * eix * P[i][k] - beta * S[k][x])          
#                     else:#no transfer learning#
#                         for k in xrange(K):
#                             P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
#                             Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])
        e_1=0
        e_2=0
        reg=0 
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    e_1 = e_1 + pow(R[i][j] - cp.dot(P[i,:],Q[:,j]), 2)
                    for k in range(K):
                        reg = reg + (beta/2) * ( pow(P[i][k],2) + pow(Q[k][j],2) )
            
#         if I !=None:
        for x in range(len(I[i])):
            if I[i][x] > 0:
                e_2 = e_2 + pow(I[i][x] - cp.dot(P[i,:],S[:,x]), 2)
                for k in range(K):
                    reg = reg + (beta/2) * ( pow(S[k][x],2) )  

        e=lamda*e_1+(1-lamda)*e_2+reg
#         else:
#             e=e_1+reg
        if e < 0.001:
            break
    print('the loss is:')
    print(e)
    return P, Q.T

def RMSE(X, Y):
    S=pow(X-Y,2)
    S=S.mean()
    return cp.sqrt(S.mean)
###############################################################################

# if __name__ == "__main__":
    
#     N3 = len(Uartp)#the row of R#

#     M3 = len(Uartp[1])#the colume of user*article#
# #    print M
#     K = 5
#     C = len(Uapp[1])#the colume of user*apps#

#     P = numpy.random.rand(N,K)
#     Q = numpy.random.rand(M,K)
#     S = numpy.random.rand(C,K)
#     nP, nQ = collective_matrix_factorization(Uartp,Uapp, P, Q, S, K)

#     nR=numpy.dot(nP,nQ.T)
#     print nR

In [34]:
def fast_double_matrix_factorization(R, I, P, Q, S, K, steps=5000, lamda=0.7, alpha=0.0002, beta=0.02): 
    R[np.isnan(R)] = 0 # Use np.isnan for csv
    I[np.isnan(I)] = 0

    r_plus_mask = np.where(R>0)
    i_plus_mask = np.where(I>0)
        
    r_plus_bin = np.sign(R)    
    i_plus_bin = np.sign(I)
    
#     IX_mask = np.repeat(np.multiply(np.sum(r_plus_bin, axis=1, keepdims=True), \
#                                     np.sum(i_plus_bin, axis=0, keepdims=True)), K, axis=1)
    
    PK_mask = np.repeat(np.sum(r_plus_bin, axis=1, keepdims=True), K, axis=1)
    PK_mask += np.repeat(np.sum(i_plus_bin, axis=1, keepdims=True), K, axis=1)
    QK_mask = np.repeat(np.sum(r_plus_bin, axis=0, keepdims=True), K, axis=0)
    SK_mask = np.repeat(np.sum(i_plus_bin, axis=0, keepdims=True), K, axis=0)
    
    
#     e = np.float64(0.0)
#     prevE = np.float64(0.0) #tmp value of loss function
    last_step = time.time()
    
    for step in range(steps):
                
        eij = R - np.dot(P,Q.T)
        eix = I - np.dot(P,S.T)
                
        eij[np.where(R <= 0)] = 0
        eix[np.where(I <= 0)] = 0
        
#         print(QK_mask.shape)
        
        P += 2 * alpha * (lamda * np.dot(eij, Q) + (1 - lamda) * np.dot(eix, S)) - alpha * beta * np.multiply(P, PK_mask)
        Q += 2 * alpha * lamda * np.dot(eij.T, P) - alpha * beta * np.multiply(Q.T, QK_mask).T
        S += 2 * alpha * (1 - lamda) * np.dot(eix.T, P) - alpha * beta * np.multiply(S.T, SK_mask).T
        
#         e1_loss = np.sum(np.power(R - np.dot(P, Q.T), 2)[r_plus_mask])
#         e2_loss = np.sum(np.power(I - np.dot(P, S.T), 2)[i_plus_mask])
        e1_loss = np.mean(np.power(R - np.dot(P, Q.T), 2)[r_plus_mask])
        e2_loss = np.mean(np.power(I - np.dot(P, S.T), 2)[i_plus_mask])
        e_reg = (beta/2) * (np.sum(np.multiply(np.power(P, 2), PK_mask)) + np.sum(np.multiply(np.power(Q.T, 2), QK_mask)) + np.sum(np.multiply(np.power(S.T, 2), SK_mask)))
#         print e_reg

        e = lamda * e1_loss + (1-lamda) * e2_loss + e_reg
                
        if step%50 == 0:
            print time.time() - last_step
            last_step = time.time()
            print step
            print e     
        
        print 'the loss ', e, 'in step ', step
        
#         if step > 0 and prevE - e < 0.01:
#             break
        
#         prevE = e
                
    print ('the loss is:')
    print (e)
    print (e1_loss , e2_loss , e_reg)
    return P, Q

In [10]:
print(type(Uart[0]))
print(type(Uapp[0]))

print(type(Uapp))
Uappf = Uapp.astype(float)

<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>


In [39]:
import time

N = len(Uartp)#the row of R#
M = len(Uartp[0])#the colume of user*article#
K = 5
C = len(Uapp[0])

P = np.random.rand(N,K)
Q = np.random.rand(M,K)
S = np.random.rand(C,K)

nP, nQ = fast_double_matrix_factorization(Uartp, Uappf, P, Q, S, K)

nR = np.dot(nP,nQ.T)
print(nR)

16.5463409424
0
73421.5334162
the loss  73421.5334162 in step  0
the loss  202275.844132 in step  1
the loss  16492128.2788 in step  2


KeyboardInterrupt: 

# Baseline 1 - Matrix Factorization

In [ ]:
#######################################
#  Baseline 1 - Matrix Factorization  #
#######################################
#!/usr/bin/python
#
# Created by Albert Au Yeung (2010)
#
# An implementation of matrix factorization
#

###############################################################################

"""
@INPUT:
    R     : a matrix to be factorized, dimension N x M
    P     : an initial matrix of dimension N x K
    Q     : an initial matrix of dimension M x K
    K     : the number of latent features
    steps : the maximum number of steps to perform the optimisation
    alpha : the learning rate
    beta  : the regularization parameter
@OUTPUT:
    the final matrices P and Q
"""
def matrix_factorization(R, P, Q, K, steps=5000, alpha=0.0002, beta=0.02):
    Q = Q.T
    for step in xrange(steps):
        for i in xrange(len(R)):
            for j in xrange(len(R[i])):
                if R[i][j] > 0:
                    eij = R[i][j] - numpy.dot(P[i,:],Q[:,j])
                    for k in xrange(K):
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])
        eR = numpy.dot(P,Q)
        e = 0
        for i in xrange(len(R)):
            for j in xrange(len(R[i])):
                if R[i][j] > 0:
                    e = e + pow(R[i][j] - numpy.dot(P[i,:],Q[:,j]), 2)
                    for k in xrange(K):
                        e = e + (beta/2) * ( pow(P[i][k],2) + pow(Q[k][j],2) )
        if e < 0.001:
            break
    return P, Q.T

###############################################################################

## Prediction 

In [ ]:
N1 = len(Uart)
M1 = len(Uart[0])
K1 = 10
P1 = numpy.random.rand(N1,K1)
Q1 = numpy.random.rand(M1,K1)
nP1, nQ1 = matrix_factorization(Uart, P1, Q1, K1)
nR1 = numpy.dot(nP, nQ.T)
print nR1

In [ ]:
nR1 = numpy.dot(nP1, nQ1.T)
print nR1
print len(nR1[0])

## Final Result

In [ ]:
topR = []
for i in range(nR1.shape[0]):
    lineR = []
    #return the index with ranked 1~5
    subRank = nR1[i].argsort()[-5:][::-1]
    print 'user #',user[i],uid[i]
#     print subRank
    for j in range(5):
        tmp = article[subRank[j]]
        lineR.append(tmp)
    print lineR
    
# print topR

In [62]:
nR = nR3.copy()

import math
topR1 = []
for i in range(nR.shape[0]):
    lineR1 = []
    lineR2 = []
    #return the index with ranked 1~5
    subRank = nR[i].argsort()[-5:][::-1]
    subRank2 = Uart[i].argsort()
    print ('user #',user[i],uid[i])
#     print subRank
    for j in range(5):
        tmp = articleP[subRank[j]]
#         tmp2 = article[subRank2[j]]
        lineR1.append(tmp)
    for k in range(nR.shape[0]):
        if math.isnan(Uart[i][subRank2[k]]):
            continue
        tmp2 = article[subRank2[k]]
        lineR2.append(tmp2)
        if len(lineR2)==5:
            break
    print (lineR1)
    print (lineR2)
    
# print topR

user # 0 00108FC5F1F3031ADD536548124A4912
['39169', '10523', '19069', '6570', '41177']
['22344']
user # 1 00310455B7B24D5CDEC892351B8D1D51
['39169', '10523', '19069', '6570', '37911']
['40351']
user # 2 00381B68ACD6CC83635CDDD9B40F0BAF
['10523', '39169', '19069', '6570', '37065']
['12897', '16891', '35968', '19071', '40613']
user # 3 003B2215B070CDB50C0819C72E208F0A
['39169', '2518', '19069', '10523', '6570']
['41089', '4367', '8597', '4283', '13976']
user # 4 003B8D414199B9C5EB7CF8C94BCC6787
['10523', '19069', '6570', '41177', '29901']
[]
user # 5 0043B1ADB92D7265FCF0B744A42AAF05
['39169', '10523', '19069', '6570', '37065']
['34968']
user # 6 00450B9C8D0055C699C03D6A0B1F6CE8
['2325', '10523', '13666', '37911', '39169']
[]
user # 7 0049D0D7726B7A1DEFB26EFE6B336B38
['10523', '39169', '19069', '6570', '41177']
[]
user # 8 004A20E76B5AFEBF06EC46D0FD33A404
['39169', '10523', '19069', '6570', '41177']
['46595', '22512', '5905', '34612', '37315']
user # 9 004A5982BCE2DECD0622672F6DFE488E
['1

In [ ]:
# topR1 = []
for i in range(nR.shape[0]):
    lineR2 = []
    #return the index with ranked 1~5
    subRank2 = Uart[i].argsort()
    print 'user #',user[i],uid[i]
    for j in range(nR.shape[0]):
        if math.isnan(Uart[i][subRank2[j]]):
            continue
        tmp2 = article[subRank2[j]]
        lineR2.append(tmp2)
        if len(lineR2)==5:
            break
    print lineR2
    
# print topR

In [ ]:
print dfArt.44347

# Baseline 2 - ?

In [ ]:
#8*5 empty
w, h = 8, 5;
Matrix = [[None for x in range(w)] for y in range(h)] 
print Matrix